# Imports

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

# Environment Data

In [2]:
#Imported data taken from the robot's readings of the entire environment.

data = [[113, 113, 149, 1662, 1368, 1326, 1521, 1521, 1138, 1154, 1154, 2146, 2550, 1399, 138, 113, 113, 135, 195, 145, 116], [113, 113, 156, 1664, 1370, 1397, 1397, 1397, 1156, 1152, 1152, 2146, 2177, 1405, 1290, 114, 114, 130, 147, 149, 113], [120, 121, 149, 1668, 1555, 1525, 1521, 1521, 1147, 1147, 1147, 2139, 2550, 1418, 1398, 114, 107, 126, 199, 203, 121], [118, 116, 162, 1672, 1381, 1527, 1520, 1520, 1160, 1160, 1160, 2150, 2550, 754, 137, 112, 112, 130, 191, 147, 120], [118, 120, 164, 1672, 1376, 1322, 1520, 1520, 1147, 1147, 532, 2154, 2550, 1406, 1280, 112, 112, 130, 157, 147, 118], [103, 114, 147, 1052, 1031, 1023, 1208, 1202, 1183, 456, 437, 435, 447, 475, 456, 435, 435, 447, 483, 504, 114], [107, 112, 145, 1046, 1030, 1022, 1217, 1206, 1160, 455, 437, 437, 452, 479, 460, 439, 439, 447, 510, 138, 112], [103, 109, 139, 1023, 1008, 1023, 1200, 1206, 1183, 456, 439, 436, 454, 492, 456, 439, 439, 454, 511, 145, 112], [109, 114, 139, 1060, 1198, 1024, 1198, 1181, 1196, 460, 439, 433, 448, 511, 464, 448, 443, 456, 479, 154, 113], [107, 112, 147, 1050, 1036, 1200, 1200, 1198, 1202, 455, 437, 437, 452, 474, 455, 439, 439, 456, 500, 523, 114], [853, 855, 869, 765, 718, 729, 729, 856, 901, 145, 103, 101, 120, 149, 810, 794, 792, 800, 833, 1191, 883], [849, 848, 893, 752, 718, 744, 849, 863, 901, 126, 107, 107, 126, 2550, 804, 788, 788, 796, 853, 1166, 865], [853, 855, 921, 921, 713, 736, 730, 865, 945, 925, 103, 101, 121, 2550, 804, 788, 786, 796, 861, 1179, 856], [857, 856, 894, 802, 728, 736, 853, 867, 941, 908, 101, 94, 120, 845, 796, 780, 778, 788, 819, 1183, 864], [856, 869, 899, 758, 714, 744, 857, 869, 950, 909, 103, 101, 121, 154, 796, 786, 779, 792, 829, 1183, 872], [103, 103, 130, 181, 139, 113, 113, 135, 947, 1997, 1825, 963, 945, 947, 784, 765, 766, 774, 804, 923, 107], [101, 107, 137, 184, 122, 109, 113, 154, 952, 1539, 1799, 948, 955, 831, 780, 765, 765, 784, 823, 126, 107], [114, 118, 143, 189, 126, 112, 107, 130, 956, 1913, 1807, 943, 941, 847, 792, 770, 770, 780, 810, 166, 120], [109, 113, 139, 184, 128, 109, 109, 128, 941, 1830, 1809, 955, 941, 933, 796, 774, 772, 778, 821, 925, 113], [103, 112, 145, 174, 122, 103, 109, 130, 972, 1821, 1769, 952, 947, 947, 788, 771, 774, 784, 827, 925, 112], [437, 433, 454, 483, 448, 433, 433, 448, 483, 483, 120, 118, 130, 162, 474, 460, 454, 462, 496, 687, 437], [429, 435, 464, 496, 452, 439, 443, 462, 326, 861, 120, 118, 137, 158, 464, 446, 443, 454, 500, 469, 435], [433, 439, 469, 490, 460, 439, 439, 455, 499, 147, 114, 112, 126, 166, 464, 448, 446, 456, 496, 484, 439], [436, 439, 464, 1192, 452, 439, 439, 460, 509, 2550, 112, 112, 121, 931, 469, 448, 448, 462, 496, 474, 439], [435, 441, 464, 483, 448, 439, 439, 456, 500, 137, 112, 112, 130, 925, 464, 446, 446, 460, 666, 471, 439], [454, 458, 475, 523, 792, 792, 773, 788, 831, 2550, 1821, 1839, 2175, 2550, 2550, 107, 107, 113, 149, 481, 460], [454, 460, 483, 515, 515, 515, 778, 794, 887, 1843, 1821, 1829, 2161, 2550, 121, 103, 103, 121, 515, 483, 455], [460, 462, 483, 532, 812, 812, 778, 794, 839, 1846, 1830, 1822, 2169, 2550, 130, 103, 103, 120, 147, 479, 462], [460, 464, 488, 525, 833, 833, 780, 796, 825, 1830, 1816, 1824, 2154, 2550, 135, 103, 103, 122, 139, 481, 464], [460, 460, 481, 537, 804, 784, 784, 800, 841, 1830, 1813, 1817, 1850, 2550, 121, 95, 95, 114, 139, 474, 462], [786, 787, 841, 326, 143, 121, 126, 173, 1288, 1259, 1280, 1468, 1488, 1857, 2550, 1353, 1634, 2550, 2550, 839, 869], [796, 802, 827, 2550, 135, 122, 121, 149, 1288, 1263, 1286, 1468, 1487, 1862, 1650, 2550, 1344, 1344, 2550, 812, 802], [796, 794, 820, 2550, 139, 113, 113, 139, 2550, 1267, 1278, 1487, 1489, 1849, 2550, 1353, 1219, 2550, 2550, 823, 796], [796, 802, 842, 2550, 130, 112, 114, 139, 1271, 1263, 1303, 1489, 1495, 1848, 1657, 1368, 1641, 1326, 2550, 818, 802], [802, 804, 843, 2550, 126, 112, 114, 138, 1269, 1255, 1297, 1488, 1495, 1850, 1641, 2090, 1252, 1353, 2550, 825, 802], [93, 91, 113, 833, 833, 1183, 1187, 871, 834, 834, 118, 113, 130, 166, 166, 114, 109, 122, 158, 183, 94], [95, 101, 113, 1164, 1164, 1187, 1187, 848, 834, 536, 112, 107, 120, 195, 143, 113, 109, 126, 181, 184, 103], [95, 101, 122, 1181, 1181, 1192, 1192, 842, 842, 137, 112, 112, 121, 175, 130, 109, 103, 126, 183, 185, 101], [93, 95, 120, 1722, 1722, 1191, 1191, 842, 849, 849, 114, 107, 126, 184, 179, 107, 107, 118, 184, 185, 101], [95, 101, 126, 1166, 1166, 1191, 1191, 863, 937, 137, 112, 112, 126, 195, 126, 107, 103, 122, 173, 184, 101], [439, 447, 464, 483, 483, 843, 841, 857, 1648, 1506, 1173, 1138, 1145, 2550, 481, 456, 454, 469, 506, 479, 452], [443, 447, 474, 500, 500, 842, 840, 856, 1680, 1210, 1147, 1135, 1143, 510, 473, 452, 452, 464, 491, 475, 447], [443, 439, 455, 487, 871, 848, 848, 856, 907, 1246, 1174, 1121, 1468, 534, 474, 447, 443, 452, 492, 496, 456], [439, 446, 464, 500, 864, 848, 848, 863, 917, 1254, 1156, 1127, 1474, 506, 464, 445, 443, 458, 474, 475, 446], [439, 446, 455, 506, 864, 853, 855, 871, 1299, 1227, 1156, 1123, 1474, 498, 455, 446, 439, 456, 474, 492, 448]]

Zone_1 = [data[0], data[1], data[2], data[3], data[4]]
Zone_2 = [data[5], data[6], data[7], data[8], data[9]]
Zone_3 = [data[10], data[11], data[12], data[13], data[14]]
Zone_4 = [data[15], data[16], data[17], data[18], data[19]]
Zone_5 = [data[20], data[21], data[22], data[23], data[24]]
Zone_6 = [data[25], data[26], data[27], data[28], data[29]]
Zone_7 = [data[30], data[31], data[32], data[33], data[34]]
Zone_8 = [data[35], data[36], data[37], data[38], data[39]]
Zone_9 = [data[40], data[41], data[42], data[43], data[44]]

# Training Environment

In [3]:
class TrainZoneEnv(Env):
    def __init__(self):
        # Actions we can take: Guess 1 of 9 zones
        self.action_space = Discrete(9)
        #  array
        self.observation_space = Box(low=np.array([0]), high=np.array([8]))
        # Set initial Zone Guess
        self.state = 0
        # Import the real Zone's sensor data
        Zone = random.randint(0,8)
        Num = 5*(Zone)
        sample = data[Num]
        self.sensor_data = sample
        # Set amount of guesses per run
        self.guess_length = 10
        
    def step(self, action):
        # Difine the sensor values for each Zone
        sensor_zone = [Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6, Zone_7, Zone_8, Zone_9]
        #apply the action
        self.state = action
        # Reduce guess length by 1
        self.guess_length -= 1 
        
        # Calculate reward
        # if the sensor data is in the chosen zone class, reward = 1
        if self.sensor_data in sensor_zone[self.state]: 
            reward = 1
            correct = True
        else: 
            reward = -1
            correct = False
        
        # Check if guessing is done
        if self.guess_length <= 0 or correct == True: 
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset Initial Zone Guess
        self.state = 0
        # Import a new Zone's sensor data
        Zone = random.randint(0,8)
        Num = 5*(Zone)
        sample = data[Num]
        self.sensor_data = sample
        # Reset Guess amount
        self.guess_length = 10
        return self.state

In [4]:
env = TrainZoneEnv()

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Zone guess:{}'.format(episode, score, n_state+1))

Episode:1 Score:1 Zone guess:3
Episode:2 Score:1 Zone guess:1
Episode:3 Score:-10 Zone guess:8
Episode:4 Score:1 Zone guess:5
Episode:5 Score:-4 Zone guess:8
Episode:6 Score:-6 Zone guess:2
Episode:7 Score:-8 Zone guess:7
Episode:8 Score:-2 Zone guess:6
Episode:9 Score:1 Zone guess:4
Episode:10 Score:-10 Zone guess:2


C:\Users\Dylan\anaconda3\lib\site-packages\gym\spaces\box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


# Model Building

In [5]:
states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
del model

NameError: name 'model' is not defined

In [7]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 9)                 225       
                                                                 
Total params: 873
Trainable params: 873
Non-trainable params: 0
_________________________________________________________________


In [8]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

# Model Training

In [9]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

C:\Users\Dylan\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 6:57 - reward: 1.0000

C:\Users\Dylan\anaconda3\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   48/10000 [..............................] - ETA: 2:00 - reward: -0.5833

C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 16 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Dylan\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 17 + 1) 

10000/10000 [==============================] - 54s 5ms/step - reward: -0.7814
1597 episodes - episode_reward: -4.890 [-10.000, 1.000] - loss: 1.503 - mae: 3.218 - mean_q: -3.210

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: -0.7770
1610 episodes - episode_reward: -4.829 [-10.000, 1.000] - loss: 1.632 - mae: 3.410 - mean_q: -3.456

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: -0.7798
1592 episodes - episode_reward: -4.893 [-10.000, 1.000] - loss: 1.664 - mae: 3.475 - mean_q: -3.546

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 59s 6ms/step - reward: -0.7642
1645 episodes - episode_reward: -4.651 [-10.000, 1.000] - loss: 1.630 - mae: 3.406 - mean_q: -3.474

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: -0.7656
done, took 287.986 seconds


# Model Testing

In [10]:
class TestZoneEnv(Env):
    def __init__(self):
        # Actions we can take: Guess 1 of 9 zones
        self.action_space = Discrete(9)
        #  array
        self.observation_space = Box(low=np.array([0]), high=np.array([8]))
        # Set initial Zone Guess
        self.state = 0
        # Import the real Zone's sensor data
        sample = data[Num]
        self.sensor_data = sample
        # Set amount of guesses per run
        self.guess_length = 10
        
    def step(self, action):
        # Difine the sensor values for each Zone
        sensor_zone = [Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6, Zone_7, Zone_8, Zone_9]
        #apply the action
        self.state = action
        # Reduce guess length by 1
        self.guess_length -= 1 
        
        # Calculate reward
        # if the sensor data is in the chosen zone class, reward = 1
        if self.sensor_data in sensor_zone[self.state]: 
            reward = 1
            correct = True
        else: 
            reward = -1
            correct = False
        
        # Check if guessing is done
        if self.guess_length <= 0 or correct == True: 
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset Initial Zone Guess
        self.state = 0
        # Import a new Zone's sensor data
        sample = data[Num]
        self.sensor_data = sample
        # Reset Guess amount
        self.guess_length = 10
        return self.state

In [49]:
# Choose the zone the Agent is in.

Zone = 7
Num = 5*(Zone -1) + 3

In [50]:
# Test to determine if the Agent knows where it is.
env = TestZoneEnv()

scores = dqn.test(env, nb_episodes=2, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 2 episodes ...
Episode 1: reward: -10.000, steps: 10
Episode 2: reward: -10.000, steps: 10
-10.0
